La finalidad de este segundo cuaderno es hacerle finetuning a un modelo basado en bert, entrenado en español (bert-base-spanish) para predecir la "dificultad" de un prompt que se le hace a nuestra herramienta "Midas Help". Así como si la pregunta no está relacionada con el TFM.

De esta forma, se optimizan costes al usar LLMs más caros (e inteligentes) unicamente cuando es necesario. Así como no llamar a ningun LLM y ofrecer una respuesta estática en caso de que se utilice para otros fines.

## 1. Importar lo necesario

Librerias necesarias etc.

In [1]:
# Celda 1: Importar dependencias
import pandas as pd
import numpy as np
import torch
import re
import copy
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import unicodedata

## 2. Generar dataset
Lo definiremos aqui mismo. Los valores de "dificultad" serán:

0 -> Pregunta "fácil"

1 -> Pregunta "dificil"

2 -> Pregunta no relacionada.

Ejemplos de preguntas serían:
* Facil: "¿cual es el enlace al repositorio github de MIDAS?"
* Dificil: "¿cuántos agentes tiene definidos midas touch? ¿cómo funciona?"
* No relacionado: "dame una receta de lasaña".

In [2]:
# Celda 2: Generar dataset

# Prompts para dificultad 0 (preguntas fáciles)
prompts_diff0 = [
    "¿Qué es Midas?",
    "¿Qué significa TFM en este contexto?",
    "¿Cuántos componentes conforman Midas?",
    "¿Para qué sirve la capa de detección de dificultad en midas help?",
    "¿Quiénes participan en el desarrollo de Midas?",
    "¿Qué objetivo tiene Midas Architech?",
    "¿En qué se diferencia Midas de un sistema convencional de ML?",
    "¿Qué lenguaje de programación se usa en Midas?",
    "¿Cual es la pagina web de Midas?",
    "¿Dónde se almacenan los repositorios de Midas?",
    "¿Se puede usar Midas con Python 3.8?",
    "¿Qué hace el componente Midas Deploy?",
    "¿Cómo ayuda Midas Assistant en el flujo de trabajo?",
    "¿Los componentes de Midas se instalan por separado?",
    "¿Cómo se integra Midas Plot con los datos?",
    "¿Cómo se han repartido los integrantes de este TFM el trabajo?",
    "¿Existen requisitos de hardware para utilizar Midas?",
    "¿De qué manera Midas Dataset genera datasets sintéticos?",
    "¿Cuáles son los pasos básicos para entrenar un modelo con Midas?",
    "¿Qué hace Midas Test?",
    "¿Puedo usar Midas con datos en formato CSV?",
    "¿Cómo se guardan los modelos entrenados en Midas?",
    "¿Hay un manual de usuario para Midas?",
    "¿Que componente MIDAS generaba las graficas?",
    "¿Cuál es la función de Midas Help?",
    "¿Dónde puedo encontrar la documentación principal de Midas?",
    "¿Qué tipo de modelos puede generar Midas?",
    "¿Cómo se activa Midas Plot desde la línea de comandos?",
    "¿La capa de clasificación de dificultad está en Midas Help?",
    "¿Puedo crear múltiples joblibs con Midas Touch?",
    "¿Se requiere alguna librería externa para Midas?",
    "¿Los agentes de Midas se comunican entre sí?",
    "¿Qué formato de dataset utiliza Midas Dataset por defecto?",
    "¿Cuándo conviene usar Midas Assistant?",
    "¿Cómo se correlacionan los datos en Midas Plot?",
    "¿Hay documentación sobre la arquitectura multiagente de Midas Touch?",
    "¿Puedo crear un modelo de clasificación con Midas?",
    "¿Está Midas orientado a la predicción o a la clasificación?",
    "¿Existen ejemplos de uso de Midas en GitHub?",
    "¿Qué tamaño de dataset es recomendable para Midas?",
    "¿En qué consiste la parte de RAG en Midas Architech?",
    "¿Cuántas fases de entrenamiento maneja Midas?",
    "¿Puedo exportar mis resultados a un archivo CSV con Midas?",
    "¿Es difícil de configurar Midas?",
    "¿Midas se basa en librerías open-source?",
    "¿Hay alguna limitación de licencia para usar Midas?",
    "¿Necesito conocimientos avanzados de ML para usar Midas?",
    "¿Qué librerías de Python utiliza Midas para su funcionamiento?",
    "¿Cómo se integra la validación cruzada en Midas?",
    "¿Puedo entrenar redes neuronales con Midas?",
    "¿Qué diferencia hay entre Midas Touch y Midas Test?",
    "¿Cómo se genera el archivo joblib al final del proceso?",
    "¿Cuál es el principal beneficio de usar Midas?",
    "¿Midas Assistant puede ayudarme a generar prompts personalizados?",
    "¿Se pueden combinar diferentes frameworks en Midas Architech?",
    "¿Qué tipo de informes produce Midas Test?",
    "¿Cómo se gestiona el control de versiones en Midas?",
    "¿Cuál es la ventaja de utilizar Midas Deploy?",
    "¿Se puede personalizar la interfaz streamlit que genera Midas Deploy?",
    "¿Qué parámetros de entrenamiento se pueden configurar en Midas Touch?",
    "¿Midas Plot muestra correlaciones estadísticas?",
    "¿Puede Midas Architech programar en varios lenguajes a la vez?",
    "¿El dataset sintético generado por Midas Dataset se puede exportar?",
    "¿Existe una forma rápida de probar el sistema Midas?",
    "¿Cómo me aseguro de que Midas Help responda correctamente?",
    "¿Qué significa convertir un CSV en oro en el contexto de Midas?",
    "¿Cuál es la mejor práctica para nombrar un proyecto Midas?",
    "¿Existen plantillas predeterminadas en Midas?",
    "¿Cómo se estructura el archivo principal de un proyecto con Midas?",
    "¿Qué ventajas tiene Midas a la hora de automatizar tareas de ML?",
    "¿Puedo entrenar un modelo de regresión con Midas?",
    "¿En qué etapa se hace el preprocesamiento de datos con Midas?",
    "¿Cómo se activa Midas Assistant para solicitar sugerencias?",
    "¿Hay ejemplos de uso práctico de Midas en la documentación?",
    "¿Es posible interrumpir el entrenamiento y reanudarlo luego en Midas?",
    "¿Cómo se gestiona la limpieza de datos en Midas?",
    "¿Dé que forma se han repartido tareas los colaboradores de Midas?",
    "¿Puedo ver un demo en vivo de Midas?",
    "¿Qué métodos de evaluación usa Midas Test?",
    "¿Midas Deploy tiene opciones para personalizar gráficos?",
    "¿Dónde puedo reportar bugs relacionados con Midas?",
    "¿Puedo añadir mis propios agentes a Midas?",
    "¿Los agentes de Midas son personalizables?",
    "¿Está Midas diseñado para entornos de producción?",
    "¿Hay una versión de Midas para Docker?",
    "¿Cómo se especifica la ruta al CSV en Midas Touch?",
    "¿Se pueden editar manualmente los datos sintéticos de Midas Dataset?",
    "¿Qué tipo de interfaz se usa para gestionar Midas?",
    "¿Cuáles son las características principales de Midas Help?",
    "¿Midas Architech soporta frameworks como Ray o Dask?",
    "¿Puedo crear scripts automáticos para ejecutar Midas en secuencia?",
    "¿Qué tamaño debería tener el archivo joblib final?",
    "¿Midas Plot puede generar gráficos en 3D?",
    "¿Hay un canal de comunicación oficial para usuarios de Midas?",
    "¿Cómo se realiza la actualización de cada componente de Midas?",
    "¿Existe un roadmap público para las futuras versiones de Midas?",
    "¿Que base de datos se utiliza en midas architech?",
    "¿Es sencillo migrar proyectos antiguos a la arquitectura Midas?",
    "¿Cómo puedo supervisar la utilización de recursos durante el entrenamiento con Midas?",
    "¿Se pueden entrenar varios modelos en paralelo con Midas?",
    "¿Tiene Midas un sistema de logging integrado?",
    "¿Existe algún ejemplo de pipeline completo con Midas?",
    "¿Cómo se configura la frecuencia de guardado de modelos en Midas?"
]

# Prompts para dificultad 1 (preguntas difíciles)
prompts_diff1 = [
    "¿Cómo funciona la comunicación interna entre los agentes de Midas para entrenar un modelo?",
    "Explica el proceso detallado de validación cruzada en Midas Test y cómo se guardan los resultados.",
    "¿De qué forma Midas Assistant puede refactorizar prompts para Midas Plot con un dataset complejo?",
    "Analiza la estructura de Midas Architech y describe cómo interactúa con la documentación RAG.",
    "¿Cuál es el algoritmo principal que usa Midas Touch para entrenar los modelos?",
    "¿Cuántos agentes define midas touch?",
    "Describe paso a paso cómo se orquesta el flujo de datos entre Midas Dataset y Midas Touch.",
    "¿Cómo gestiona Midas la generación de múltiples datasets sintéticos con diferentes distribuciones estadisticas?",
    "Explica la relación entre la capa de embeddings y la capa de detección de dificultad en Midas Help.",
    "¿De qué manera Midas Plot genera graficos atractivos en datasets de alta dimensión?",
    "¿Cómo se integran las capacidades de RAG en la arquitectura de Midas Architech para la búsqueda de información?",
    "¿Cuál es la metodología de optimización por hiperparámetros que utiliza Midas Touch?",
    "Analiza los posibles cuellos de botella en la fase de despliegue con Midas Deploy.",
    "¿Cómo se realiza la extracción de características automáticas en Midas Touch antes de entrenar un modelo?",
    "¿Qué mecanismos de versionado emplea Midas para sincronizar actualizaciones en varios componentes?",
    "¿Cómo decide Midas Assistant qué componente recomendar ante una petición compleja del usuario?",
    "Explícame en detalle la arquitectura multiagente de Midas y cómo cada agente coordina sus tareas.",
    "¿Es posible utilizar embeddings contextuales en Midas Plot para agrupar datos no numéricos?",
    "Describe el proceso de escalabilidad de Midas cuando se trabaja con grandes volúmenes de datos.",
    "¿Cuál es la lógica interna de Midas Help para decidir si una pregunta es fácil, difícil o no relacionada?",
    "¿De qué forma se aplican métodos de explicabilidad (Explainable AI) en los modelos generados por Midas?",
    "Analiza el tratamiento de outliers en Midas Dataset y su impacto en el posterior entrenamiento.",
    "¿Cómo interactúan Midas Test y Midas Assistant para refinar los parámetros de validación cruzada?",
    "¿Cúal es el llm usado en midas?",
    "Explica el uso de GPU en Midas Touch y cómo afecta el rendimiento del entrenamiento.",
    "¿Qué estrategias utiliza Midas Deploy para manejar múltiples usuarios concurrentes?",
    "¿De qué manera se realiza la limpieza y normalización avanzada de datos en Midas Dataset?",
    "¿Cómo se implementan las técnicas de bagging o boosting dentro de la arquitectura Midas?",
    "Describe cómo Midas Assistant puede generar prompts que aprovechen la API de Midas Plot.",
    "¿Qué consideraciones de seguridad se han tomado en cuenta para Midas Help y su RAG?",
    "Explica la lógica tras la selección automática de hiperparámetros en Midas Touch.",
    "¿Cómo se manejan las inconsistencias entre los componentes si uno de ellos falla durante el entrenamiento?",
    "¿De qué manera se podría integrar un módulo de autoML dentro de la arquitectura de Midas?",
    "Explica cómo Midas Test crea los informes de calidad y qué métricas son las más relevantes.",
    "¿Cuál es la aproximación de Midas Architech para identificar marcos teóricos en la documentación?",
    "¿Cómo se implementa el control de versiones en la base de datos de Midas Dataset?",
    "Analiza la posibilidad de utilizar MLOps con Midas. ¿Qué pipelines se podrían automatizar?",
    "¿De qué manera Midas Assistant decide qué tipo de ayuda ofrecer basándose en la consulta del usuario?",
    "¿En qué escenarios la generación de datos sintéticos de Midas Dataset puede inducir sesgos en el modelo?",
    "Describe a fondo el método de correlación que utiliza Midas Plot para datos categóricos.",
    "¿Cómo se gestionan las excepciones cuando Midas Touch no logra converger en un modelo estable?",
    "¿De qué modo se integran metodologías de Continuous Integration en Midas Test?",
    "¿Es posible configurar Midas para que use distintos frameworks de ML (e.g., TensorFlow, PyTorch) en paralelo?",
    "Explica cómo Midas Deploy genera la interfaz de Streamlit y cómo personalizar sus componentes.",
    "¿Qué tan modular es Midas Architech para adaptar nuevas fuentes de documentación RAG?",
    "¿Cómo se implementan las recomendaciones de prompts que realiza Midas Assistant?",
    "Analiza las ventajas e inconvenientes de utilizar Midas Plot para visualizaciones 3D complejas.",
    "¿Qué nivel de granularidad permite Midas Help a la hora de categorizar preguntas de usuarios?",
    "¿Cómo maneja Midas la verificación de la calidad de datos en tiempo real durante el entrenamiento?",
    "¿De qué forma Midas Assistant interactúa con Midas Test para sugerir nuevas estrategias de prueba?",
    "Explica cómo Midas Touch aprovecha la arquitectura multiagente para acelerar el proceso de training.",
    "¿Cómo se integran técnicas de NLP en Midas Help para entender las preguntas del usuario?",
    "¿Qué pipeline se sigue en Midas Architech para incorporar nuevas definiciones a la documentación?",
    "Analiza la forma de persistir y cargar metadatos en los joblib generados por Midas Touch.",
    "¿Existen planes para incorporar técnicas de reinforcement learning en Midas?",
    "Describe un escenario complejo donde Midas Assistant deba coordinar varios componentes a la vez.",
    "¿Cómo se evalúa la robustez de los agentes si existen inputs maliciosos en Midas Help?",
    "¿Qué técnicas de oversampling o undersampling se aplican automáticamente en Midas Dataset?",
    "Explica cómo se podría extender Midas Deploy para integrar un servicio de autenticación de usuarios.",
    "¿De qué manera Midas Plot logra manejar datos de alta dimensionalidad sin colapsar la visualización?",
    "Describe la importancia de los embeddings en el análisis semántico de Midas Help.",
    "¿Cómo se conectan las salidas de Midas Test con Midas Assistant para la toma de decisiones?",
    "Explica el proceso interno de Midas Architech para indexar y recuperar documentación específica.",
    "¿Qué tipo de análisis estadístico avanzado puede realizar Midas Plot aparte de las correlaciones?",
    "¿Cómo se integraría un orquestador externo (como Airflow) con la arquitectura de Midas?",
    "¿Existe algún método para monitorear el uso de memoria de cada agente en Midas en tiempo real?",
    "¿Qué tan viable es incluir análisis de series temporales dentro de Midas Touch?",
    "Analiza la forma en que Midas Assistant infiere la intención del usuario para sugerir el componente adecuado.",
    "Explica el proceso de fusión de datasets sintéticos con datos reales en Midas Dataset.",
    "¿Cómo se implementa el logging distribuido en la arquitectura multiagente de Midas?",
    "¿De qué forma se manejan las credenciales de acceso cuando Midas Architech consulta documentación externa?",
    "¿Cómo adaptar Midas Plot para escenarios con datos geoespaciales?",
    "¿Qué tipo de criterios utiliza Midas Test para decidir la métrica principal de evaluación?",
    "Explica la interconexión entre Midas Deploy y Midas Touch en un entorno productivo.",
    "¿De qué modo Midas Assistant detecta inconsistencias en los prompts generados por los usuarios?",
    "Analiza la escalabilidad horizontal de Midas cuando hay múltiples cargas de entrenamiento simultáneas.",
    "¿Cómo se sincronizan los cambios en la configuración de Midas con cada uno de sus componentes?",
    "¿Qué técnicas de reducción de dimensionalidad admite Midas Plot para mejorar la visualización?",
    "Explica los pasos para incorporar un nuevo agente personalizado al ecosistema de Midas.",
    "¿Cómo se maneja la compatibilidad de versiones entre Midas Architech y Midas Touch?",
    "¿Es posible añadir nuevos docs a midas architech?",
    "Analiza las formas de mejorar la precisión en la capa de detección de dificultad de Midas Help.",
    "¿Qué mecanismos de caching utiliza Midas para acelerar la generación de datasets sintéticos?",
    "¿De qué manera Midas Test automatiza la comparación de diferentes modelos entrenados?",
    "Explica cómo Midas Assistant gestiona la experiencia conversacional con diferentes usuarios simultáneos.",
    "¿Qué tan flexible es la estructura de Midas Deploy para integrar librerías de visualización externas?",
    "¿Cómo se podria utilizar Transfer Learning en la capa de embeddings de Midas Help?",
    "¿Qué ocurre si Midas Plot detecta colinealidad extrema entre las variables de un dataset?",
    "Analiza la factibilidad de incorporar un agente para limpieza avanzada de texto en Midas Dataset.",
    "¿Cómo podríamos escalar el sistema Midas a una arquitectura en la nube con Kubernetes?",
    "¿Qué protocolos de seguridad se deben seguir para exponer la interfaz de Midas Deploy a internet?",
    "Explica en detalle cómo Midas Touch maneja la hibridación de varios algoritmos de ML en un mismo pipeline.",
    "¿De qué forma Midas Architech se asegura de mantener actualizada su base de conocimiento RAG?",
    "¿Cómo se implementa la actualización incremental de un modelo entrenado en Midas Touch?",
    "Analiza la precisión de la clasificación que realiza la capa de dificultad en Midas Help.",
    "¿Qué métodos de auto-documentación existen para registrar cada uno de los pasos en Midas?",
    "¿Cómo Midas Assistant genera prompts que aprovechan la correlación de Midas Plot para mejorar un dataset?",
    "Explica la complejidad de integrar datos no estructurados (como texto) en Midas Dataset.",
    "¿Se pueden definir reglas personalizadas de preprocesamiento en Midas Touch?",
    "¿Qué mecanismos de rollback existen si una actualización de Midas falla en producción?",
    "Analiza la efectividad de la orquestación multiagente en Midas cuando hay varios datasets grandes.",
    "¿Cómo mide Midas Test la estabilidad de los resultados en validaciones repetidas?",
    "Explica de forma detallada cómo cada componente de Midas encaja en el ciclo de vida del ML."
]

# Prompts para dificultad 2 (cosas no relacionadas)
prompts_diff2 = [
    "Dame una receta de lasaña",
    "¿Cuál es el significado oculto del número 42 en la cultura pop?",
    "Recomiéndame una película de ciencia ficción para ver hoy",
    "¿Cuál es el mejor método para cocinar pasta al dente?",
    "Explica por qué los gatos suelen temerle al agua",
    "¿Qué opinas de la música clásica en la era digital?",
    "Necesito consejos para plantar un huerto urbano en mi balcón",
    "¿Quién ganará la próxima copa mundial de fútbol?",
    "¿Por qué las nubes son blancas y el cielo es azul?",
    "Dame tu teoría sobre la existencia de vida extraterrestre",
    "¿Qué libro de fantasía épica recomiendas para leer?",
    "¿Cuál es la forma más rápida de aprender un idioma nuevo?",
    "Comparte una anécdota divertida sobre perros y pelotas",
    "¿Por qué algunas personas prefieren el té en lugar del café?",
    "Dime la receta más rara que conozcas que incluya chocolate",
    "¿Crees que los robots conquistarán el mundo algún día?",
    "¿Cuál es tu opinión sobre la criogenización de humanos?",
    "Quiero saber qué hay en el centro de la Tierra",
    "¿Por qué la gente dice que el dinero no da la felicidad?",
    "Convénceme de que el helado de vainilla es superior al de chocolate",
    "¿Es posible entrenar un pez para hacer trucos?",
    "Explícame la trama de la película Inception en un minuto",
    "¿Qué opinas del arte abstracto y sus interpretaciones?",
    "¿Cómo se puede enseñar matemáticas a un niño pequeño?",
    "Necesito un tip para organizar mi escritorio de oficina",
    "Dime un poema breve inspirado en la naturaleza",
    "¿Cuál es la ciudad más hermosa del mundo para visitar?",
    "¿Existen los fantasmas o son pura invención humana?",
    "¿Qué opinas sobre las dietas cetogénicas?",
    "¿Quién fue el mejor guitarrista de la historia del rock?",
    "Recomiéndame una serie de televisión para maratonear este fin de semana",
    "¿Cuál es el valor nutricional de las semillas de chía?",
    "Dime un chiste sobre elefantes",
    "¿Cuál es tu teoría favorita de la formación del universo?",
    "Convénceme de que las ardillas son las verdaderas dueñas de los parques",
    "¿Para qué sirve aprender a hacer origami?",
    "¿Cómo puedo mejorar mi postura mientras trabajo en la computadora?",
    "Describe un paisaje otoñal usando solo adjetivos",
    "¿Cuál es la estación del año más romántica y por qué?",
    "Necesito ejemplos de comidas veganas rápidas y sabrosas",
    "Cuéntame algo curioso sobre la historia del ajedrez",
    "¿Prefieres un libro físico o un eBook? Justifica tu respuesta",
    "¿Por qué crees que las jirafas tienen el cuello tan largo?",
    "Recomienda una rutina de ejercicios para principiantes",
    "¿Qué opinas de la moda retro que está regresando?",
    "Haz una reseña breve de tu videojuego favorito",
    "¿Cuál es el secreto para preparar sushi en casa?",
    "Argumenta a favor o en contra del uso de la tecnología en las aulas",
    "¿Dónde se originó el tango y por qué es tan popular?",
    "Dime un mito popular sobre la nutrición que no sea cierto",
    "¿Qué tipo de perro es mejor para una familia con niños?",
    "¿Cómo podríamos reducir el uso de plásticos en la vida diaria?",
    "Necesito recomendaciones para meditar y aliviar el estrés",
    "¿Cuál es la clave para entender la poesía contemporánea?",
    "Defiende la idea de que la pizza con piña es deliciosa",
    "¿Qué opinas de la moda de pintar murales en la ciudad?",
    "Dame un consejo para combatir la procrastinación",
    "¿Cómo influye la música en el estado de ánimo de las personas?",
    "¿Por qué se celebra el Día de la Tierra y qué objetivos persigue?",
    "Haz un top 3 de postres tradicionales de tu país",
    "¿Cuál fue la primera civilización en desarrollar la escritura?",
    "Explica cómo funciona el reciclaje de vidrio",
    "¿Qué deportes extremos recomiendas para los más arriesgados?",
    "¿Qué te parece la astrología? ¿Crees en el horóscopo?",
    "Dame un consejo para mejorar la expresión oral en público",
    "¿Cuál es el mejor truco de magia que has visto?",
    "Explica brevemente por qué la noche es oscura",
    "¿Qué tipo de pintura es mejor para un lienzo: óleo o acrílico?",
    "¿Por qué la gente colecciona objetos como monedas o sellos?",
    "¿Qué opinas de la evolución de la inteligencia artificial en videojuegos?",
    "Dime algo interesante sobre la cultura japonesa",
    "¿Crees que el minimalismo es la clave de la felicidad?",
    "Recomienda una estrategia para ahorrar más dinero cada mes",
    "¿Cuál es la mejor forma de aprender a tocar la guitarra?",
    "¿Qué significa que un evento deportivo sea 'amateur'?",
    "¿Por qué los pingüinos viven en zonas tan frías?",
    "Define el término 'serendipia' con un ejemplo real",
    "¿Puede la meditación ayudar a mejorar el rendimiento académico?",
    "Dime tu opinión sobre la música electrónica y sus subgéneros",
    "¿Es viable la construcción de ciudades subterráneas en el futuro?",
    "¿Por qué algunas personas creen en teorías conspirativas absurdas?",
    "Dame un ejemplo de ejercicio de respiración para relajarse",
    "¿Cuál es tu festividad favorita y por qué la celebras?",
    "¿Cómo se originó la tradición de los fuegos artificiales?",
    "Explica la importancia de la biodiversidad en el ecosistema",
    "¿Qué es la acidificación de los océanos y cómo nos afecta?",
    "¿Crees que la música instrumental es mejor para concentrarse?",
    "¿Cómo se fabrica el papel reciclado a gran escala?",
    "Haz una lista de ventajas y desventajas de vivir en una gran ciudad",
    "¿Cuál es tu personaje de ficción favorito y por qué?",
    "Dame un dato curioso sobre la cultura egipcia antigua",
    "¿Qué te parece la idea de colonizar Marte en las próximas décadas?",
    "Explica qué es la nanotecnología de forma sencilla",
    "¿Existe vida después de la muerte? Comparte tu punto de vista",
    "¿Cuáles son los fundamentos del pensamiento crítico?",
    "Narra una historia de terror muy breve pero inquietante",
    "¿Por qué la comida picante produce esa sensación de ardor?",
    "¿Crees que los humanos heredaremos la Tierra o la destruiremos?",
    "¿Cuál es la clave para mantener una buena relación de pareja?",
    "Dime tres ideas para personalizar una fiesta de cumpleaños"
]

# --- por aqui terminamos los prompts ---

# Construcción del dataset añadiendo la dificultad correspondiente
datos = []

for prompt in prompts_diff0:
    datos.append({"prompt": prompt, "dificultad": 0})

for prompt in prompts_diff1:
    datos.append({"prompt": prompt, "dificultad": 1})

for prompt in prompts_diff2:
    datos.append({"prompt": prompt, "dificultad": 2})

# Creamos el DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(datos)

# Guardamos el DataFrame como un archivo CSV
df.to_csv("dataset_raw.csv", index=False)

print("El dataset.csv ha sido creado y guardado exitosamente")

El dataset.csv ha sido creado y guardado exitosamente


## 3. Preprocesar el texto

De esta forma no dará problemas al entrenarse.

In [3]:
# Celda 3: Preprocesamiento de texto
def clean_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

df = pd.read_csv("dataset_raw.csv")
df['cleaned_prompt'] = df['prompt'].apply(clean_text)
texts = df['cleaned_prompt'].tolist()
labels = df['dificultad'].tolist()

Dividimos el dataset en test y train

In [4]:
# Celda 4: División del dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42, shuffle=True
)

Y lo tokenizamos usando el tokenizador propio del modelo a utilizar. Con su longitud maxima de 512 tokens.

In [5]:
# Celda 5: Tokenización
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

Creamos los dataloaders (para procesar el dataset por lotes)

In [6]:
# Celda 6: Creación de DataLoaders
class MIDASDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MIDASDataset(train_encodings, train_labels)
test_dataset = MIDASDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Configuramos el modelo y las labels...

In [7]:
# Celda 7: Configuración del modelo
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=3
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 4. Entrenar el modelo

Aqui definimos los parametros del entrenamiento...

In [8]:
# Celda 8: Configuración del entrenamiento
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Y ahora entrenamos al modelo. Serán 5 epocas y se guardará aquel que obtenga un menor "loss" en el conjunto de test. De esta forma nos quedaremos con aquel que tenga un mejor desempeño sin caer en el sobreajuste.

In [9]:
# Celda 9: Entrenamiento
best_val_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    # Entrenamiento
    model.train()
    total_train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs, labels=batch['labels'].to(device))
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # Evaluación en el conjunto de test (o validación)
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = model(**inputs, labels=batch['labels'].to(device))
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)

    print(f"Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")

    # Guardar el mejor modelo según el loss en el conjunto de validación
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"--> Mejor modelo actualizado en Epoch {epoch+1}")

# Cargar el estado del mejor modelo
model.load_state_dict(best_model_state)
print("Entrenamiento completado. Se cargó el modelo con mejor desempeño en validación.")

Epoch 1:   0%|          | 0/31 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 31/31 [02:05<00:00,  4.04s/it]


Epoch 1 - Train Loss: 0.7145 - Validation Loss: 0.3571
--> Mejor modelo actualizado en Epoch 1


Epoch 2: 100%|██████████| 31/31 [01:53<00:00,  3.66s/it]


...

Epoch 3 - Train Loss: 0.0677 - Validation Loss: 0.1877
--> Mejor modelo actualizado en Epoch 3


Epoch 4: 100%|██████████| 31/31 [01:45<00:00,  3.41s/it]


Epoch 4 - Train Loss: 0.0216 - Validation Loss: 0.3292


Epoch 5: 100%|██████████| 31/31 [01:45<00:00,  3.40s/it]


Epoch 5 - Train Loss: 0.0112 - Validation Loss: 0.2502
Entrenamiento completado. Se cargó el modelo con mejor desempeño en validación.


Vamos a sacar el reporte de clasificación para ver el desempeño

In [10]:
# Celda 10: Evaluación
model.eval()
predictions, true_labels = [], []

for batch in tqdm(test_loader, desc="Evaluando"):
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(batch['labels'].cpu().numpy())

print("\nReporte de clasificación:")
print(classification_report(true_labels, predictions))

Evaluando: 100%|██████████| 8/8 [00:05<00:00,  1.37it/s]


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        21
           1       0.95      0.90      0.93        21
           2       1.00      1.00      1.00        20

    accuracy                           0.95        62
   macro avg       0.95      0.95      0.95        62
weighted avg       0.95      0.95      0.95        62



Muy buen resultado. Quizas un pelin bajo el recall de "pregunta dificil" (comparado con los otros), pero es una MUY buena mejora frente a bgem3+xgboost.

Ahora, guardamos el modelo y el tokenizador.

In [11]:
# Celda 11: Guardar modelo
model.save_pretrained("prompt_analysis")
tokenizer.save_pretrained("prompt_analysis")

('prompt_analysis/tokenizer_config.json',
 'prompt_analysis/special_tokens_map.json',
 'prompt_analysis/vocab.txt',
 'prompt_analysis/added_tokens.json')

Y vamos a probar algunos ejemplos:

In [ ]:
# Celda 12: Función de predicción
def clasificar_dificultad(texto):
    # Preprocesar
    texto_limpio = clean_text(texto)

    # Tokenizar
    inputs = tokenizer(texto_limpio, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Predecir
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediccion = torch.argmax(logits, dim=1).item()

    return prediccion

# Ejemplo de uso
while True:
    user_input = input("\nIngrese su pregunta (o 'salir' para terminar): ")
    if user_input.lower() == 'salir':
        break
    dificultad = clasificar_dificultad(user_input)
    print(f"Dificultad clasificada: {dificultad}")


Ingrese su pregunta (o 'salir' para terminar): en que consiste este tfm midas
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): que modelos llm se usan en midas touch
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): cual es la logica de la comunicacion entre agentes en midas touch
Dificultad clasificada: 1

Ingrese su pregunta (o 'salir' para terminar): seria posible añadir una nueva fuente de docs a midas architech?
Dificultad clasificada: 1

Ingrese su pregunta (o 'salir' para terminar): necesito gafas nuevas
Dificultad clasificada: 2

Ingrese su pregunta (o 'salir' para terminar): dame una receta de lasaña midas
Dificultad clasificada: 2

Ingrese su pregunta (o 'salir' para terminar): cuantas personas han trabajado en este tfm?
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): salir


Estos resultados si que me convencen mucho mas respecto a los anteriores de BGE-M3 + XGBoost.

Estamos listos para integrarlo en Midas Help.